In [1]:
import numpy as np
import string
import codecs
from itertools import combinations

# Hidden Markov Models

Here we implement a hidden markov model and test it on various language datasets

In [150]:
class HMM:
    def __init__(self,A=None,B=None,pi=None):
        self.A = A
        self.B = B
        self.pi = pi
    
    def _forward(self,obs):
        """
        Computes the scaled forward probability matrix and scaling factors.
        
        Parameters:
            obs : ndarray of shape (T,)  The observation sequence
            
        Returns:
            alpha : ndarray of shape (T,N) 
                The scaled forward probability matrix
            c : ndarray of shape (T,)
                The scaling factors c = [c_1,c_2,...,c_T]
        """
        #initialize things
        T,N = obs.shape[0],B.shape[1]
        alpha = np.zeros((T,N))
        c = np.zeros(T)
        #base case
        c[0] = np.dot(self.pi,self.B[obs[0],:])**-1
        alpha[0,:] = c[0]*self.pi*self.B[obs[0],:]
        #compute the rest of alpha and c
        for t in range(1,T):
            a = self.A@alpha[t-1,:]
            c[t] = np.dot(a,self.B[obs[t],:])**-1
            alpha[t,:] = c[t]*(a*self.B[obs[t],:])
        return alpha,c
    
    def _backward(self, obs, c):
        """
        Computes the scaled backward probability matrix.
        Parameters:
            obs : ndarray of shape (T,) The observation sequence
            c : ndarray of shape (T,) 
                The scaling factors from the forward pass
                
        Returns:
            beta : ndarray of shape (T,N)
                The scaled backward probability matrix
        """
        #initialize things
        T,N = obs.shape[0],B.shape[1]
        beta = np.zeros((T,N))
        beta[-1,:] = c[-1]
        
        for t in reversed(range(T-1)):
            b=(self.B[obs[t+1],:]*beta[t+1,:])
            beta[t,:] = c[t]*self.A.T@b
        
        return beta
    
    def _delta(self, obs, alpha, beta):
        """
        Computes the delta probabilities.
        
        Parameters:
            obs : ndarray of shape (T,) The observation sequence
            alpha : ndarray of shape (T,N)
                The scaled forward probability matrix from the forward pass
            beta : ndarray of shape (T,N)
                The scaled backward probability matrix from the backward pass
        
        Returns:
            delta : ndarray of shape (T-1,N,N)  The delta probability array
            gamma : ndarray of shape (T,N)  The gamma probability array
        """
        T,N = alpha.shape
        delta = np.zeros((T-1,N,N))
        gamma = np.zeros((T,N))
        
        for t in range(T-1):
            for i in range(N):
                for j in range(N):
                    top = alpha[t,i]*self.A[j,i]*self.B[obs[t+1],j]*beta[t+1,j]
                    bottom = np.sum(np.sum(alpha[t,:]*self.A,axis=1)*
                                        self.B[obs[t+1],:]*
                                        beta[t+1,:])
                    delta[t,i,j] = top/bottom
                gamma[t,i] = sum(delta[t,i])
            
        gamma[-1,:] = np.multiply(alpha[-1,:],beta[-1,:])/np.dot(alpha[-1,:],beta[-1,:])
        return delta,gamma
    
    def _estimate(self, obs, delta, gamma):
        """
        Estimates better parameter values.
        Parameters:
            obs : ndarray of shape (T,)  The observation sequence
            delta : ndarray of shape (T-1,N,N) The delta probability array
            gamma : ndarray of shape (T,N)  The gamma probability array
        """
        # update self.A, self.B, self.pi in place
        Ia,Ja = self.A.shape
        Ib,Jb = self.B.shape
        T = obs.shape[0]
        
        self.pi = gamma[0,:]
        for i in range(Ia):
            for j in range(Ja):
                self.A[i,j] = np.sum(delta[:-1,j,i])/np.sum(gamma[:-1,j])
                
        for i in range(Ib):
            for j in range(Jb):
                self.B[i,j] = np.sum(gamma[:,j][obs==i])/np.sum(gamma[:,j])
                
        return self
    
    def fit(self,obs, A=None, B=None, pi=None, max_iter=100, tol=1e-3):
        """
        Fits the model parameters to a given observation sequence.
        Parameters:
            obs : ndarray of shape (T,) 
                Observation sequence on which to train the model.
            A : stochastic ndarray of shape (N,N)
                Initialization of state transition matrix
            B : stochastic ndarray of shape (M,N)
                Initialization of state observation matrix
            pi : stochastic ndarray of shape (N,)
                Initialization of initial state distribution
            max_iter : integer  The maximum number of iterations to take
            tol : float  The convergence threshold for change in log-probability
        """
        # initialize self.A, self.B, self.pi
        # run the iteration
        if A is not None:
            self.A = A
        if B is not None:
            self.B = B
        if pi is not None:
            self.pi = pi
        if self.A is None or self.B is None or self.pi is None:
            raise ValueError("A, B, and pi must all have a value")
        log_prob1 = sum(-(np.log(self._forward(obs)[1])))
        for i in range(max_iter):
            alpha,c = self._forward(obs)
            beta = self._backward(obs,c)
            delta,gamma = self._delta(obs,alpha,beta)
            self._estimate(obs,delta,gamma)
            log_prob2 = sum(-(np.log(self._forward(obs)[1])))
            if np.linalg.norm(log_prob1-log_prob2) < tol:
                break
            log_prob1=log_prob2
        return log_prob2
            

# Declaration

We will try it on the declaration of independence

In [160]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)
def prep_data(filename):
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower() #and convert to all lower case
    # remove punctuation and newlines
    remove_punct_map = {ord(char): None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)
    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    #make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}
    #convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)
    return symbols, obs_sequence

symbols, obs = prep_data('declaration.txt')
M = len(set(obs))
N = 2
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = HMM()
h.fit(obs,A,B,pi,200)
for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1]))                

 , 0.1545, 0.1895
a, 0.0682, 0.0441
b, 0.0022, 0.0352
c, 0.0299, 0.0087
d, 0.0461, 0.0000
e, 0.1273, 0.0692
f, 0.0000, 0.0762
g, 0.0238, 0.0000
h, 0.0639, 0.0000
i, 0.0822, 0.0000
j, 0.0000, 0.0068
k, 0.0026, 0.0000
l, 0.0160, 0.0596
m, 0.0153, 0.0255
n, 0.0884, 0.0000
o, 0.0317, 0.1438
p, 0.0000, 0.0584
q, 0.0000, 0.0025
r, 0.0283, 0.1144
s, 0.0674, 0.0465
t, 0.1102, 0.0159
u, 0.0075, 0.0711
v, 0.0135, 0.0000
w, 0.0157, 0.0048
x, 0.0013, 0.0008
y, 0.0032, 0.0269
z, 0.0007, 0.0000


In [161]:
M = len(set(obs))
N = 3
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = HMM()
h.fit(obs,A,B,pi,200)
for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1]))                

 , 0.0301, 0.1258
a, 0.0000, 0.0703
b, 0.0146, 0.0277
c, 0.0398, 0.0374
d, 0.0848, 0.0075
e, 0.0000, 0.0000
f, 0.0546, 0.0147
g, 0.0413, 0.0080
h, 0.1169, 0.0000
i, 0.0000, 0.0322
j, 0.0058, 0.0000
k, 0.0040, 0.0006
l, 0.0601, 0.0311
m, 0.0408, 0.0159
n, 0.0458, 0.1807
o, 0.0000, 0.0835
p, 0.0288, 0.0295
q, 0.0022, 0.0000
r, 0.1113, 0.0594
s, 0.1127, 0.0789
t, 0.1352, 0.1349
u, 0.0000, 0.0411
v, 0.0268, 0.0000
w, 0.0217, 0.0187
x, 0.0018, 0.0021
y, 0.0197, 0.0000
z, 0.0014, 0.0000


In [163]:
M = len(set(obs))
N = 4
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = HMM()
h.fit(obs,A,B,pi,200)
for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1]))                

 , 0.0000, 0.7358
a, 0.2041, 0.0007
b, 0.0095, 0.0000
c, 0.0334, 0.0154
d, 0.0020, 0.0000
e, 0.1181, 0.0221
f, 0.0124, 0.0096
g, 0.0195, 0.0000
h, 0.0000, 0.0000
i, 0.0901, 0.1361
j, 0.0052, 0.0023
k, 0.0017, 0.0000
l, 0.0000, 0.0009
m, 0.0000, 0.0127
n, 0.0000, 0.0000
o, 0.1771, 0.0000
p, 0.0363, 0.0191
q, 0.0000, 0.0016
r, 0.0000, 0.0230
s, 0.0306, 0.0109
t, 0.1575, 0.0000
u, 0.0856, 0.0054
v, 0.0000, 0.0046
w, 0.0169, 0.0000
x, 0.0000, 0.0000
y, 0.0000, 0.0000
z, 0.0000, 0.0000


Looks like the hidden states change the outcome!

# War and Peace

Now we will try it on the Russian version of War and Peace

In [164]:
symbols, obs = prep_data('WarAndPeace.txt')
M = len(set(obs))
N = 2
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = HMM()
h.fit(obs,A,B,pi,200)
for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1]))                

 , 0.2145, 0.0879
а, 0.0000, 0.1761
б, 0.0250, 0.0000
в, 0.0655, 0.0000
г, 0.0296, 0.0000
д, 0.0385, 0.0000
е, 0.0186, 0.1418
ж, 0.0140, 0.0000
з, 0.0252, 0.0000
и, 0.0016, 0.1316
й, 0.0149, 0.0000
к, 0.0498, 0.0010
л, 0.0719, 0.0000
м, 0.0381, 0.0000
н, 0.0973, 0.0000
о, 0.0000, 0.2408
п, 0.0352, 0.0054
р, 0.0597, 0.0000
с, 0.0514, 0.0279
т, 0.0779, 0.0000
у, 0.0000, 0.0590
ф, 0.0018, 0.0003
х, 0.0111, 0.0000
ц, 0.0049, 0.0000
ч, 0.0167, 0.0038
ш, 0.0109, 0.0000
щ, 0.0047, 0.0000
ъ, 0.0003, 0.0003
ы, 0.0000, 0.0376
ь, 0.0001, 0.0445
э, 0.0000, 0.0066
ю, 0.0079, 0.0024
я, 0.0128, 0.0328
ё, 0.0000, 0.0001


In [165]:
M = len(set(obs))
N = 3
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T
pi = np.random.dirichlet(np.ones(N))
h = HMM()
h.fit(obs,A,B,pi,200)
for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}".format(symbols[i], h.B[i,0], h.B[i,1]))                

 , 0.4126, 0.0000
а, 0.0000, 0.0215
б, 0.0193, 0.0307
в, 0.0482, 0.0623
г, 0.0331, 0.0192
д, 0.0364, 0.0362
е, 0.0021, 0.0288
ж, 0.0139, 0.0120
з, 0.0334, 0.0078
и, 0.0000, 0.0323
й, 0.0000, 0.0000
к, 0.0343, 0.0637
л, 0.0858, 0.0379
м, 0.0000, 0.0614
н, 0.0685, 0.1300
о, 0.0000, 0.0449
п, 0.0000, 0.0968
р, 0.0346, 0.0920
с, 0.0652, 0.0494
т, 0.0561, 0.1017
у, 0.0000, 0.0065
ф, 0.0019, 0.0020
х, 0.0134, 0.0052
ц, 0.0049, 0.0043
ч, 0.0156, 0.0224
ш, 0.0144, 0.0035
щ, 0.0053, 0.0031
ъ, 0.0000, 0.0012
ы, 0.0000, 0.0124
ь, 0.0000, 0.0000
э, 0.0000, 0.0109
ю, 0.0000, 0.0000
я, 0.0010, 0.0000
ё, 0.0000, 0.0000


Looks like the vowels are the one that looks like 'a', backwards 'in' symbol, bl, y, o, e, and backwards 'N'